## 실습: select_model
---
Sklearn 모델 추천 함수 활용
- sklearn.utils 모듈의 all_estimators(type_filter)
    * type_filter 파라미터: 'classifier','regressor' 지정
    * 반환: 해당 타입의 모델 리스트 => 모델이름, 모델객체

In [1]:
# 모듈 로딩
from sklearn.utils import all_estimators
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

##### [1] 데이터 로딩 및 확인

In [2]:
# Bunch 데이터 타입 => dict와 유사한 형태
data =load_iris()

In [3]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
# data, target => numpy 타입
# target_names => 라벨 => setosa, versicolor...
# feature_names => 컬럼명
input_data = data['data']
input_target = data['target']

##### [2] 학습 데이터 분리

In [5]:
# 학습 - 테스트 데이터셋
train_input, test_input, train_target, test_target =train_test_split(input_data, input_target, test_size = 0.2, random_state = 42)

In [6]:
# 학습 - 검증용 데이터셋
train_input, val_input, train_target, val_target =train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

##### [3] 학습
- 학습방법 선정 => 분류/회귀
    - 분류 => knn, logisticRegression, DecisionTreeClassifier, SGDClassifier,SVC...
    - 분류모델 적용 후결과 => all_estimators()

In [7]:
# 필터 타입에 해당하는 sklearn에 존재하는 모든 모델 이름과 객체 리스트로 반환
models = all_estimators(type_filter = 'classifier')
models

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('ClassifierChain', sklearn.multioutput.ClassifierChain),
 ('ComplementNB', sklearn.naive_bayes.ComplementNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('GaussianProcessClassifier',
  sklearn.gaussian_process._gpc.GaussianProcessClassifier),
 ('GradientBoostingClassifier',
  sklearn.ensemble._gb.GradientBoostingClassifier),
 ('HistGradientBoostingClassifier',
  sklearn.ensemble._hist_gradi

In [8]:
# 각 모델들 훈련시키고 정확도 추출
scores = []
for name, model in models:
    try:
        # 모델 객체 생성
        md = model()
        # 학습
        md.fit(train_input, train_target)
        # 평가
        result = md.score(test_input, test_target)
    
        scores.append((name, result))
    except:
        pass

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

In [9]:
# 각 분류기 모듈명과 score
scores

[('AdaBoostClassifier', 1.0),
 ('BaggingClassifier', 1.0),
 ('BernoulliNB', 0.36666666666666664),
 ('CalibratedClassifierCV', 0.9),
 ('CategoricalNB', 0.9666666666666667),
 ('ComplementNB', 0.7),
 ('DecisionTreeClassifier', 0.9333333333333333),
 ('DummyClassifier', 0.36666666666666664),
 ('ExtraTreeClassifier', 0.9666666666666667),
 ('ExtraTreesClassifier', 1.0),
 ('GaussianNB', 1.0),
 ('GaussianProcessClassifier', 1.0),
 ('GradientBoostingClassifier', 1.0),
 ('HistGradientBoostingClassifier', 0.9666666666666667),
 ('KNeighborsClassifier', 0.9666666666666667),
 ('LabelPropagation', 1.0),
 ('LabelSpreading', 1.0),
 ('LinearDiscriminantAnalysis', 1.0),
 ('LinearSVC', 1.0),
 ('LogisticRegression', 1.0),
 ('LogisticRegressionCV', 1.0),
 ('MLPClassifier', 0.9666666666666667),
 ('MultinomialNB', 0.7666666666666667),
 ('NearestCentroid', 0.9666666666666667),
 ('NuSVC', 1.0),
 ('PassiveAggressiveClassifier', 0.8),
 ('Perceptron', 0.9333333333333333),
 ('QuadraticDiscriminantAnalysis', 0.966666

## K-Fold Cross Validation 
---
- 가장 일반적으로 사용되는 교차 검증 방법. 
- 보통 회귀 모델에 사용, 데이터가 독립적이고 동일한 분포 가진 경우 사용

- 훈련 데이터가 줄어드는 문제 및 데이터가 충분하지 않은 문제 해결
- 테스트 데이터에 과대적합(Overfitting) 문제 해결 
- 훈련 데이터를 동일 크기로 여러 조각 나눈 후 데이터를교차시켜훈련/검증 데이터로 활용

##### [장점] 
- 모든 데이터셋을 훈련과 평가에 활용 가능=> 정확도 ▲  
- 데이터부족 과소적합 방지
- 평가용 데이터 편중 ▼ 
- 평가 결과에 일반화된 모델 생성

##### [단점] 
- 훈련과 평가에 많은 시간 소요

- cross_validate()함수 사용
    - 기본값 cv = 5-Fold ===>모델 5개

In [13]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import numpy as np

In [20]:
# iris 데이터 불러오기
data = load_iris()

# data, target 데이터 받기
input_data = data['data']
input_target = data['target']

# data, target 형태 확인
input_data.shape, input_target.shape

((150, 4), (150,))

In [21]:
input_data.shape, input_target.shape

((150, 4), (150,))

In [28]:
# 모델 객체 생성
model_LR = LogisticRegression(max_iter=500)

In [32]:
# 5-Fold로 5등분으로 나누어서 학습/검증 모델 생성 진행
result = cross_validate(model_LR, input_data, input_target, return_train_score= True)

In [30]:
result

{'fit_time': array([0.02950907, 0.0341084 , 0.08168006, 0.04094005, 0.02259016]),
 'score_time': array([0.00086188, 0.00104499, 0.00055957, 0.00040317, 0.00040197]),
 'test_score': array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ]),
 'train_score': array([0.96666667, 0.96666667, 0.98333333, 0.98333333, 0.975     ])}

In [36]:
print(result['test_score'].mean())
print(result['train_score'].mean())

0.9733333333333334
0.975
